In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from whiteGPT.utils.data.gpt_dataset import AkutagawaSampleDataset as Dataset
import MeCab

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, context_size, d_model):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(context_size, d_model)
        for pos in range(context_size):
            for i in range(0, d_model, 2):
                pe[pos,i]   = math.sin(pos/(10000**((2*i)/d_model)))
                pe[pos,i+1] = math.cos(pos/(10000**((2*i)/d_model)))
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return self.pe[:, :x.size(1)].detach()

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = nn.Dropout(0.1)

    def forward(self, q, k, v, mask):
        a = torch.matmul(q, k.transpose(2, 3)) / (d_model ** 0.5)
        if mask is not None:
            a = a.masked_fill(mask == 0, float("-inf"))
        a = F.softmax(a, dim=-1)
        a = self.dropout(a)
        a = torch.matmul(a, v)
        return a

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        self.fc_q = nn.Linear(d_model, d_model)
        self.fc_k = nn.Linear(d_model, d_model)
        self.fc_v = nn.Linear(d_model, d_model)
        self.attention = ScaledDotProductAttention()
        self.fc = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        N, S, H, D = q.size(0), q.size(1), n_head, d_model // n_head
        q = self.fc_q(q).view(N, S, H, D).transpose(1, 2)
        k = self.fc_k(k).view(N, S, H, D).transpose(1, 2)
        v = self.fc_v(v).view(N, S, H, D).transpose(1, 2)
        attn = self.attention(q, k, v, mask=mask)
        attn = attn.transpose(1, 2).contiguous().view(N, S, -1)
        attn = self.dropout(self.fc(attn))
        return attn

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_model * 4 )
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(d_model * 4, d_model)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = self.dropout(self.fc2(x))
        return x

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention()
        self.ff = FeedForward(d_model)

    def forward(self, x, mask=None):
        q = self.norm_1(x)
        x = self.attn(q, q, q, mask) + x # 残差結合
        x = self.ff(self.norm_2(x)) + x # 残差結合
        return x

In [ ]:
class GPT(nn.Module):
    def __init__(self, context_size, d_model):
        super(GPT, self).__init__()
        self.context_size = context_size
        self.d_model = d_model
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(context_size, d_model)
        self.dropout = nn.Dropout(0.1)
        self.transformer_block = nn.ModuleList(
            [TransformerBlock() for _ in range(n_block)])
        self.norm = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model * context_size, vocab_size)

    def forward(self, x, mask=None):
        x = self.token_embedding(x) + self.positional_encoding(x)
        x = self.dropout(x)
        for block in self.transformer_block:
            x = block(x, mask)
        x = self.norm(x)
        x = x.view(-1, self.context_size * self.d_model)
        x = self.fc(x)
        return x

In [ ]:
def create_attention_mask(s):
    return (torch.triu(torch.ones((s, s)),1) == 0) * 1

In [ ]:
context_size = 10 
vocab_size = 10   
d_model = 256 
n_head = 1 
n_block = 1 

In [ ]:
model = GPT(context_size, d_model)
x = [1,1,1,1,1,1,1,1,1,1]
x = torch.LongTensor([x])
y = model(x)

In [ ]:
y

tensor([[-0.1089,  0.7643,  0.0663,  0.9351,  0.1751, -0.4679, -0.2348, -0.1751,
         -0.4797, -0.1268]], grad_fn=<AddmmBackward0>)

In [ ]:
y.argmax()

tensor(3)

In [ ]:
dataset = Dataset()
dataset.tagger = MeCab.Tagger("-Owakati")
model_path = '/content/whiteGPT/model/akutagawa/pre-trained.pkl' #@param{type:'string'}
model = torch.load(model_path, map_location='cpu', weights_only=False)
print('ok.')

ok.


In [ ]:
## 芥川龍之介モデルで生成
n_token = 500
sentence = '良平は一瞬間呆気にとられた。'
dataset.sample(model, sentence, n=n_token, t=0.1, use_topk=False)

良平は一瞬間呆気にとられた。もうかれこれ暗くなる事、去年の暮母と岩村まで来たが、今日の途はその三四倍ある事、それを今からたった一人、歩いて帰らなければならない事、――そう云う事が一時にわかったのである。良平は殆ど泣きそうになった。が、泣いても仕方がないと思った。泣いている場合ではないとも思った。彼は若い二人の土工に、取って附けたような御時宜をすると、どんどん線路伝いに走り出した。良平は少時無我夢中に線路の側を走り続けた。その内に懐の菓子包みが、邪魔になる事に気がついたから、それを路側へ抛り出す次手に、板草履も其処へ脱ぎ捨ててしまった。すると薄い足袋の裏へじかに小石が食いこんだが、足だけは遙かに軽くなった。彼は左に海を感じながら、急な坂路を駈け登った。時時涙がこみ上げて来ると、自然に顔が歪んで来る。――それは無理に我慢しても、鼻だけは絶えずくうくう鳴った。竹藪の側を駈け抜けると、夕焼けのした日金山の空も、もう火照りが消えかかっていた。良平は、愈気が気でなかった。往きと返りと変るせいか、景色の違うのも不安だった。すると今度は着物までも、汗の濡れ通ったのが気になったから、やはり必死に駈け続けたなり、羽織を路側へ脱いで捨てた。蜜柑畑へ来る頃には、あたりは暗くなる一方だった。「命さえ助かれば――」良平はそう思いながら、辷ってもつまずいても走って行った。やっと遠い夕闇の中に、村外れの工事場が見えた時、良平は一思いに泣きたくなった。しかしその時もべそはかいたが、とうとう泣かずに駈け続けた。彼の村へはいって見ると、もう両側の家家には、電燈の光がさし合っていた。良平はその電燈の光に、頭から汗の湯気の立つのが、彼自身にもはっきりわかった。井戸端に水を汲んでいる女衆や、畑から帰って来る男衆は、良平が喘ぎ喘ぎ走るのを見

In [ ]:
n_token = 1000
sentence = '遺族？じゃこの画を描いた人は'
dataset.sample(model, sentence, n=n_token, t=0.1, use_topk=False)


遺族？　じゃこの画を描いた人は死んでいるのですか。」「死んでいるのです。もっとも生きている中から、死んだようなものでしたが。」私の好奇心はいつか私の不快な感情より強くなっていた。「どうして？」「この画描きは余程前から気が違っていたのです。」「この画を描いた時もですか。」「勿論です。気違いででもなければ、誰がこんな色の画を描くものですか。それをあなたは傑作だと云って感心してお出でなさる。そこが大に面白いですね。」記者はまた得意そうに、声を挙げて笑った。彼は私が私の不明を恥じるだろうと予測していたのであろう。あるいは一歩進めて、鑑賞上における彼自身の優越を私に印象させようと思っていたのかも知れない。しかし彼の期待は二つとも無駄になった。彼の話を聞くと共に、ほとんど厳粛にも近い感情が私の全精神に云いようのない波動を与えたからである。私は悚然として再びこの沼地の画を凝視した。そうして再びこの小さなカンヴァスの中に、恐しい焦躁と不安とに虐まれている傷しい芸術家の姿を見出した。そうしてあらゆる優れた芸術品から受ける様に、この黄いろい沼地の草木からも恍惚たる悲壮の感激を受けた。実際同じ会場に懸かっている大小さまざまな画の中で、この一枚に拮抗し得るほど力強い画は、どこにも見出す事が出来なかったのである。「大へんに感心していますね。」こう云う言と共に肩を叩かれた私は、あたかも何かが心から振い落されたような気もちがして、卒然と後をふり返った。「どうです、これは。」相手は無頓着にこう云いながら、剃刀を当てたばかりの顋で、沼地の画をさし示した。流行の茶の背広を着た、恰幅の好い、消息通を以て自ら任じている、――新聞の美術記者である。私はこの記者から前にも一二度不快な印象を受けた覚えがあるので、不承不承に返事をした。「傑作です。」「傑作――ですか。これは面白い。」記者は腹を揺って笑った。その声に驚かされたのであろう。近くで画を見ていた二三人の見物が皆云い合せたようにこちらを見た。私はいよいよ不快になった。「これは面白い。元来この画はね、会員の画じゃないのです。が、何しろ当人が口癖のようにここへ出す出すと云っていたものですから、遺族が審査員へ頼んで、やっとこの隅へ懸ける事になったのです。」「遺族？じゃこの画を描いた人は死んでいるのですか。」「死んでいるのです。もっとも生きている中から、死んだよう